In [63]:
import numpy as np
import pandas as pd
import os

In [64]:
#Obtain the county file names, in a list, from the State parent folder
#Takes the parent directory (string) as parameter
#returns the list containing all the file names

def get_filenames(statename):
    parentfolder = "C:/Users/Anthony DePinho/Documents/REU 2017/TRiCAM_BostonAQ/StateData"+ "/" + statename
    filenames = []
    filenames = os.listdir(parentfolder)
    return filenames

In [65]:
#Concatenate Data frames

def df_county_concatenation(statename):
    statepath = "C:/Users/Anthony DePinho/Documents/REU 2017/TRiCAM_BostonAQ/StateData"+ "/" + statename
    countydfs = []
    countylist = []
    countylist = os.listdir(statepath)
    for x in range(0, len(countylist)):
        df1 = pd.read_csv(statepath + "/" + countylist[x])
        countydfs.append(df1)
    result = pd.concat(countydfs)
    return result
    

In [146]:
'''takes in a pandas dataframe with column "Units of Measure" and "Arithmetic Mean", and replaces ml/m3 into ppm in the original dataframe'''
def replace_units(df):
    tf = df["Units of Measure"] == ("Micrograms/cubic meter (LC)" or "Micrograms/cubic meter (25 C)")
    df['Arithmetic Mean'] = np.where(tf, df['Arithmetic Mean']*0.001, df['Arithmetic Mean'])

In [200]:
state = "Washington"
counties = get_filenames(state)
print(counties)

['Adams.csv', 'Asotin.csv', 'Benton.csv', 'Chelan.csv', 'Clallam1.csv', 'Clallam2.csv', 'Clark.csv', 'Columbia.csv', 'Cowlitz.csv', 'Douglas.csv', 'Grant.csv', 'Grays_Harbor1.csv', 'Grays_Harbor2.csv', 'Jefferson.csv', 'King1.csv', 'King2.csv', 'Kitsap.csv', 'Kittitas1.csv', 'Kittitas2.csv', 'Klickitat.csv', 'Lewis1.csv', 'Lewis2.csv', 'Mason.csv', 'Okanogan1.csv', 'Okanogan2.csv', 'Pierce1.csv', 'Pierce2.csv', 'Skagit.csv', 'Snohomish1.csv', 'Snohomish2.csv', 'Snohomish3.csv', 'Spokane.csv', 'Stevens1.csv', 'Stevens2.csv', 'Thurston1.csv', 'Thurston2.csv', 'Walla_Walla.csv', 'Whatcom1.csv', 'Whatcom2.csv', 'Whitman1.csv', 'Whitman2.csv', 'Yakima1.csv', 'Yakima2.csv']


In [201]:
fullstate = df_county_concatenation(state)

In [202]:
fullstate.to_csv("C:/Users/Anthony DePinho/Documents/REU 2017/TRiCAM_BostonAQ/StateData"+ "/" + state + ".csv")
print("file written")

file written


In [203]:
#After the files are written, we must read in the pandas dataframe and make appropriate conversions into ppm
replace_units(fullstate)
fullstate.shape

(1413, 57)

In [204]:
yes = 0
for string in fullstate["Parameter Name"].unique():
    if "2.5" in string:
        yes+=1
        print(string + "\n")
print(yes)

Acceptable PM2.5 AQI & Speciation Mass

PM2.5 - Local Conditions

Arsenic PM2.5 LC

Aluminum PM2.5 LC

Bromine PM2.5 LC

Calcium PM2.5 LC

Chromium PM2.5 LC

Copper PM2.5 LC

Chlorine PM2.5 LC

Iron PM2.5 LC

Lead PM2.5 LC

Manganese PM2.5 LC

Nickel PM2.5 LC

Magnesium PM2.5 LC

Phosphorus PM2.5 LC

Selenium PM2.5 LC

Titanium PM2.5 LC

Vanadium PM2.5 LC

Silicon PM2.5 LC

Zinc PM2.5 LC

Strontium PM2.5 LC

Sulfur PM2.5 LC

Rubidium PM2.5 LC

Potassium PM2.5 LC

Sodium PM2.5 LC

Zirconium PM2.5 LC

Chloride PM2.5 LC

Total Nitrate PM2.5 LC

OC PM2.5 LC TOR

EC PM2.5 LC TOR

OC1 PM2.5 LC

OC2 PM2.5 LC

OC3 PM2.5 LC

OC4 PM2.5 LC

OP PM2.5 LC TOR

EC1 PM2.5 LC

EC2 PM2.5 LC

EC3 PM2.5 LC

Nitrite PM2.5 LC

Ammonium Sulfate PM2.5 LC

Ammonium Nitrate PM2.5 LC

Soil PM2.5 LC

Organic Carbon Mass PM2.5 LC

Sulfate PM2.5 LC

Antimony PM2.5 LC

Barium PM2.5 LC

Cadmium PM2.5 LC

Cobalt PM2.5 LC

Cerium PM2.5 LC

Cesium PM2.5 LC

Indium PM2.5 LC

Tin PM2.5 LC

Silver PM2.5 LC

Ammonium Ion PM

In [205]:
print((fullstate.iloc[[0]]))

   State Code  County Code  Site Num  Parameter Code  POC  Latitude  \
0          53            1         3           11203    1     47.12   

   Longitude  Datum Parameter Name Sample Duration  \
0  -118.3819  WGS84  Light scatter          1 HOUR   

                         ...                         10th Percentile  \
0                        ...                                     0.0   

        Local Site Name                     Address  State Name County Name  \
0  RITZVILLE - ALDER ST  109 W ALDER, RITZVILLE, WA  Washington       Adams   

   City Name    CBSA Name  Date of Last Change              Site_keys  \
0  Ritzville  Othello, WA           2017-02-27  [  47.12   -118.3819]   

                                                  LU  
0  {'Residential': 0.13, 'Industrial': 0.01, 'Cro...  

[1 rows x 57 columns]


In [206]:
micropercubic = fullstate.loc[fullstate['Units of Measure'] == "Micrograms/cubic meter (LC)"]
micropercubic["Parameter Name"].unique()
print(micropercubic.shape)

(885, 57)


In [207]:
local = micropercubic.loc[micropercubic['Parameter Name'] == "PM2.5 - Local Conditions"]
localconditions = local.loc[local["Pollutant Standard"] == 'PM25 Annual 2012']
localconditions["Arithmetic Mean"]

15     0.003868
20     0.006699
16     0.006649
134    0.007709
217    0.006534
387    0.005420
392    0.005458
507    0.005565
4      0.004196
5      0.004156
10     0.009438
54     0.012100
13     0.006810
29     0.006798
33     0.007886
38     0.006510
13     0.004696
18     0.005901
4      0.006187
9      0.006358
4      0.003984
4      0.005687
16     0.007988
21     0.007584
8      0.004865
6      0.012143
24     0.008676
29     0.008604
105    0.009723
110    0.010334
6      0.012143
Name: Arithmetic Mean, dtype: float64

In [208]:
localconditions["Method Name"].unique()

array([nan, 'Multiple Methods Used',
       'R & P Model 2025 PM2.5 Sequential w/WINS - GRAVIMETRIC',
       'R & P Model 2025 PM-2.5 Sequential Air Sampler w/VSCC - Gravimetric'], dtype=object)

In [209]:
localconditions_filtereddf = localconditions.loc[:, ["Parameter Name", "Latitude", "Longitude", "LU", "Arithmetic Mean"]]
localconditions_filtereddf

,Parameter Name,Latitude,Longitude,LU,Arithmetic Mean
15,PM2.5 - Local Conditions,47.430610,-120.341950,{'Commercial and Services': 0.0680272108843537...,0.003868
20,PM2.5 - Local Conditions,47.430610,-120.341950,{'Commercial and Services': 0.0680272108843537...,0.006699
16,PM2.5 - Local Conditions,45.643360,-122.587370,"{'Commercial and Services': 0.05, 'Residential...",0.006649
134,PM2.5 - Local Conditions,47.597222,-122.319722,{'Commercial and Services': 0.3918918918918919...,0.007709
217,PM2.5 - Local Conditions,47.559750,-122.338265,{'Commercial and Services': 0.0384615384615384...,0.006534
387,PM2.5 - Local Conditions,47.568236,-122.308628,{'Commercial and Services': 0.0511945392491467...,0.005420
392,PM2.5 - Local Conditions,47.568236,-122.308628,{'Commercial and Services': 0.0511945392491467...,0.005458
507,PM2.5 - Local Conditions,47.386111,-122.230278,"{'Commercial and Services': 0.61, 'Residential...",0.005565
4,PM2.5 - Local Conditions,47.592675,-122.627397,"{'Commercial and Services': 0.61, 'Residential...",0.004196
5,PM2.5 - Local Conditions,46.993640,-120.545000,"{'Commercial and Services': 0.43, 'Residential...",0.004156


In [210]:
localconditions_filtereddf.to_csv("C:/Users/Anthony DePinho/Documents/REU 2017/TRiCAM_BostonAQ/StateData"+ "/" + state + "_localconditions.csv")
print("file written")

file written


In [152]:
otherpm25 = micropercubic.loc[micropercubic['Parameter Name'] != "PM2.5 - Local Conditions"]

In [153]:
otherpm25["Parameter Name"].unique()

array(['PM10 - LC', 'Antimony PM2.5 LC', 'Arsenic PM2.5 LC',
       'Aluminum PM2.5 LC', 'Barium PM2.5 LC', 'Bromine PM2.5 LC',
       'Cadmium PM2.5 LC', 'Calcium PM2.5 LC', 'Chromium PM2.5 LC',
       'Cobalt PM2.5 LC', 'Copper PM2.5 LC', 'Chlorine PM2.5 LC',
       'Cerium PM2.5 LC', 'Cesium PM2.5 LC', 'Iron PM2.5 LC',
       'Lead PM2.5 LC', 'Indium PM2.5 LC', 'Manganese PM2.5 LC',
       'Nickel PM2.5 LC', 'Magnesium PM2.5 LC', 'Phosphorus PM2.5 LC',
       'Selenium PM2.5 LC', 'Tin PM2.5 LC', 'Titanium PM2.5 LC',
       'Vanadium PM2.5 LC', 'Silicon PM2.5 LC', 'Silver PM2.5 LC',
       'Zinc PM2.5 LC', 'Strontium PM2.5 LC', 'Sulfur PM2.5 LC',
       'Rubidium PM2.5 LC', 'Potassium PM2.5 LC', 'Sodium PM2.5 LC',
       'Zirconium PM2.5 LC', 'Ammonium Ion PM2.5 LC',
       'Sodium Ion Pm2.5 LC', 'Potassium Ion PM2.5 LC',
       'Total Nitrate PM2.5 LC', 'OC PM2.5 LC TOR', 'EC PM2.5 LC TOR',
       'OC1 PM2.5 LC', 'OC2 PM2.5 LC', 'OC3 PM2.5 LC', 'OC4 PM2.5 LC',
       'OP PM2.5 LC TO